# Lecture Notebook: Big Data and Graph Data

Apache Spark is a big data engine that runs on compute clusters, including on the cloud.  This notebook is set up assuming that (1) Spark is running on a cloud server that is public and (2) we need to run the actual Python commands on that server, requiring us to put `%%spark` "magic" commands at the start of each cell.

You may wish to look at this notebook first, without directly running it, until you understand how to launch your own Spark cluster.


In [0]:
!apt install libkrb5-dev
!pip install sparkmagic

In [0]:
%load_ext sparkmagic.magics

The following line connects to Spark running remotely.  You will need to change the URL after the `-u` to connect to an active server. Check the instruction on how to set up EMR cluster on AWS if you are not aware of how to do it. 

In [0]:
%spark add -s my_session -l python -u http://ec2-18-213-115-72.compute-1.amazonaws.com:8998 
# The above can connect to an EMR node running Spark + Livy, assuming the firewall is set to let anyone in

In [0]:
%spark delete -s my_session_2

## Example of Loading Sharded Data (Lecture Slides A)

Some preliminaries:  **Every** cell in this notebook will need `%%spark` at the start so it runs on the remote machine with Spark instead of on the machine with Jupyter.

In [0]:
%%spark
import json
# import requests
import urllib.request as urllib

In [0]:
%%spark
import sys
print(sys.version)

In [0]:
%%spark
# 10K records from linkedin
# linked_in = requests.get('X')
# If you have problems with the above, use urllib 
linked_in = urllib.urlopen('https://XXX/test_data_10000.json')

# my_list = [json.loads(line) for line in linked_in.iter_lines()]
# If you have problems with the the above, use urllib 
my_list = [json.loads(line) for line in linked_in.readlines()]
len(my_list)


## Load the list into Spark

Spark needs to know the structure of the data in its dataframes, i.e., their schemas.  Since our JSON structure for LinkedIn is complex, we need to define the schema.

There are some basic types:
  * The table is a `StructType` with a list of fields (each row)
  * Most fields, in our case, are `StringType`.
  * We also have nested dictionary for the name, which is a `MapType` from `StringType` keys to `StringType` values.
  * `skills` is an `ArrayType` since it's a list, and it contains `StringType`s.
  * `also_view` is an array of structs.

See Pyspark documentation on `StructType` and examples such as https://www.programcreek.com/python/example/104715/pyspark.sql.types.StructType.

In [0]:
%%spark

# Spark requires that we define a schema for the LinkedIn data...
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, MapType
schema = StructType([
        StructField("_id", StringType(), True),
        StructField("name", MapType(StringType(), StringType()), True),
        StructField("locality", StringType(), True),
        StructField("skills", ArrayType(StringType()), True),
        StructField("industry", StringType(), True),
        StructField("summary", StringType(), True),
        StructField("url", StringType(), True),
        StructField("also_view", ArrayType(\
                    StructType([\
                      StructField("url", StringType(), True),\
                      StructField("id", StringType(), True)])\
                    ), True)\
         ])

In [0]:
%%spark
# Load the remote data as a list of dictionaries
linked_df = sqlContext.createDataFrame(my_list, schema).\
  repartition('_id')

linked_df.show(5)

In [0]:
%%spark
linked_df.rdd.getNumPartitions()

In [0]:
%%spark
linked_df.filter(linked_df.locality == 'United States')[['_id', 'name', 'locality']].show(5)

In [0]:
%%spark
linked_df.select("_id", "locality").show(5)

In [0]:
%%spark
### Clean out the list from memory
my_list = []

In [0]:
%%spark
linked_df.createOrReplaceTempView('linked_in')
sqlContext.sql('select * from linked_in').show(5)

In [0]:
%%spark
sqlContext.sql('select _id, name.given_name, name.family_name from linked_in').show(5)

In [0]:
%%spark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

acro = udf(lambda x: ''.join([n[0] for n in x.split()]), StringType())

linked_df.select("_id", acro("locality").alias("acronym")).show(5)

In [0]:
%%spark
# Which industries are most popular?
sqlContext.sql('select count(_id), industry '+\
               'from linked_in '+\
               'group by industry '+\
               'order by count(_id) desc').\
    show(5)

## Graphs 

For the next set of examples, we will look at graph-structured data.  It turns out our LinkedIn dataset has a list of nodes (by int ID, but associated with the user ID we used in the linked_in table) and a list of edges.

In [0]:
%%spark

import urllib.request as urllib
import zipfile
import os

#url = 'E'
url = 'https://XXX/linkedin.edges.zip'
filehandle, _ = urllib.urlretrieve(url)

zip_file_object = zipfile.ZipFile(filehandle, 'r')
fname = zip_file_object.open('linkedin.edges')

edges = []
MAX = 2000000

for link in fname:
  edge = link.decode('utf-8').split(' ')
  edges.append([int(edge[0]), int(edge[1])])
  if len(edges) >= MAX:
    break


In [0]:
%%spark

from pyspark.sql.types import IntegerType
schema = StructType([
        StructField("from", IntegerType(), True),
        StructField("to", IntegerType(), True)
         ])
# Load the remote data as a list of dictionaries
edges_df = sqlContext.createDataFrame(edges, schema)

edges_df.show(5)

In [0]:
%%spark
edges_df.createOrReplaceTempView('edges')
sqlContext.sql('select from as id, count(to) as degree from edges group by from').show(5)

## Traversing the Graph

In [0]:
%%spark

from pyspark.sql.functions import col

# Start with a subset of nodes
start_nodes_df = edges_df[['from']].filter(edges_df['from'] < 100000).\
  select(col('from').alias('id')).drop_duplicates()

start_nodes_df.show(5)

# The neighbors require us to join
# and we'll use Spark DataFrames syntax here
neighbor_nodes_df = start_nodes_df.\
  join(edges_df, start_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_nodes_df.show(5)

In [0]:
%%spark
edges_df[['from']].orderBy('from').drop_duplicates().show()

edges_df.filter(edges_df['from'] == 698).show()

In [0]:
%%spark
neighbor_neighbor_nodes_df = neighbor_nodes_df.\
  join(edges_df, neighbor_nodes_df.id == edges_df['from']).\
  select(col('to').alias('id'))

neighbor_neighbor_nodes_df.show(5)

In [0]:
%%spark
def iterate(df, depth):
  df.createOrReplaceTempView('iter')

  # Base case: direct connection
  result = sqlContext.sql('select from, to, 1 as depth from iter')

  for i in range(1, depth):
    result.createOrReplaceTempView('result')
    result = sqlContext.sql('select r1.from as from, r2.to as to, r1.depth+1 as depth  '\
                            'from result r1 join iter r2 '\
                            'on r1.to=r2.from')
  return result

In [0]:
%%spark
iterate(edges_df.filter(edges_df['from'] < 1000000), 1).orderBy('from','to').show()

In [0]:
%%spark
iterate(edges_df.filter(edges_df['from'] < 1000000), 2).orderBy('from','to').show()

In [0]:
%%spark
iterate(edges_df.filter(edges_df['from'] < 1000000), 3).orderBy('from','to').show()

In [0]:
%%spark
# Clear list of edges from Python memory
# to free up space
edges = []


### Now let's get the list of node IDs
#url = 'N'
url = 'https://XXX/linkedin.nodes.new.zip'
nodehandle, _ = urllib.urlretrieve(url)

zip_file_object = zipfile.ZipFile(nodehandle, 'r')
fname = zip_file_object.open('linkedin.nodes.new')

nodes = []
MAX = 100000

for node in fname:
  node_tuple = node.split()
  nodes.append([int(node_tuple[0]), str(node_tuple[1])])
  if len(nodes) >= MAX:
    break


## Joins in Spark, Beyond Graph Traversals

What if we want to connect our edges to the people from our previous crawl?  Sadly the edges use int node IDs that don't correspond to the people dataframe.  But in fact the node data includes this information, so let's load and exploit that.

Let's load the information about nodes, and their correspondence to the user ID.

In [0]:
%%spark
schema = StructType([
        StructField("nid", IntegerType(), True),
        StructField("user", StringType(), True)
         ])
# Load the remote data as a list of dictionaries
nodes_df = sqlContext.createDataFrame(nodes, schema)

nodes_df.show(5)

## Finding Friends, by ID

In [0]:
%%spark

nodes_df.createOrReplaceTempView('nodes')
edges_df.createOrReplaceTempView('edges')

friends_df = \
sqlContext.sql('select n1.user, n2.user as friend ' +\
               'from (nodes n1 join edges e on n1.nid = e.from) join nodes n2 on e.to = n2.nid')

friends_df.show(5)


## Connecting Friends to Names

In [0]:
%%spark
friends_df.createOrReplaceTempView('friends')

sqlContext.sql('select u1.name.given_name as user, u2.name.given_name as friend '+\
               'from (linked_in u1 join friends on u1._id = user) join linked_in u2 on u2._id = friend').show(5)